# climpy — Date reale: AMO, Tripole, corelații

**Flux:** SST ERSSTv5 → EOF → AMO + Tripole → corelații SAT / SLP / Precipitații

Rulează celulele **în ordine**.

In [ ]:
# ── Instalare pachete ───────────────────────────────────────────────────────
import subprocess, sys
subprocess.run([sys.executable, '-m', 'pip', 'install', '-q',
                'cartopy', 'eofs', 'xarray', 'numpy', 'pandas',
                'matplotlib', 'scipy', 'netCDF4', 'cmocean'], check=True)
import os
repo_root = os.path.expanduser('~')
if repo_root not in sys.path:
    sys.path.insert(0, repo_root)
import climpy
print(f'climpy {climpy.__version__} — OK')

In [ ]:
# ── Descărcare date (~2 min) ────────────────────────────────────────────────
import os
os.makedirs('data/real', exist_ok=True)
datasets = {
    'data/real/sst.nc'   : 'https://downloads.psl.noaa.gov/Datasets/noaa.ersst.v5/sst.mnmean.nc',
    'data/real/sat.nc'   : 'https://downloads.psl.noaa.gov/Datasets/gistemp/combined/250km/air.2x2.250.mon.anom.comb.nc',
    'data/real/precip.nc': 'https://downloads.psl.noaa.gov/Datasets/gpcc/full_v2020/precip.mon.total.1x1.v2020.nc',
    'data/real/slp.nc'   : 'https://downloads.psl.noaa.gov/Datasets/ncep.reanalysis.derived/surface/slp.mon.mean.nc',
}
for dest, url in datasets.items():
    if os.path.exists(dest):
        print(f'  checkmark {dest} — deja descărcat ({os.path.getsize(dest)//1024//1024} MB)')
    else:
        print(f'  arrow {dest} ...')
        os.system(f'wget -q --show-progress -O {dest} {url}')
        print(f'    done — {os.path.getsize(dest)//1024//1024} MB')
print('Gata.')

In [ ]:
# ── Imports ─────────────────────────────────────────────────────────────────
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import cmocean.cm as cmo
import climpy

%matplotlib inline
plt.rcParams['figure.dpi'] = 120
climpy.use_style('nature')

DATA = Path('data/real')
os.makedirs('figures', exist_ok=True)

CMAP_SST    = cmo.balance
CMAP_SAT    = cmo.balance
CMAP_PRECIP = cmo.tarn
CMAP_SLP    = cmo.curl
print('OK')

## 1 — Preprocessing SST

In [ ]:
sst_r = climpy.preprocess(
    DATA / 'sst.nc',
    var            = 'sst',
    lon_convention = '[-180,180]',
    fill_value     = -9.96921e+36,
    time           = ('1854-01', '2019-12'),
    lat            = (0, 80),
    lon            = (-76, 20),
    ref_period     = ('1951-01', '1980-12'),
    compute_annual = True,
    subtract_gm    = True,
)
annual_sst = sst_r['annual_no_gm'].dropna('year', how='all')
print('SST:', dict(annual_sst.sizes))
print('Ani:', int(annual_sst.year[0]), '–', int(annual_sst.year[-1]))

## 2 — EOF

In [ ]:
solver = climpy.EOF(annual_sst, min_variance=70)
solver.summary()

In [ ]:
eofs = solver.eofs()
pcs  = solver.pcs()
frac = solver.variance_fraction()

# ── Diagnostic: arată EOF1 și EOF2 raw ──────────────────────────────────────
import cartopy.feature as cfeature
fig, axes = plt.subplots(1, 2, figsize=(10, 3),
    subplot_kw={'projection': climpy.Map()})
for ax, mode, title in zip(axes,
        [eofs.isel(mode=0), eofs.isel(mode=1)],
        ['EOF 1 (raw)', 'EOF 2 (raw)']):
    data = mode.values
    vm = np.nanmax(np.abs(data))
    ax.set_extent([-76, 20, 0, 80])
    ax.add_feature(cfeature.LAND.with_scale('110m'), facecolor='#2E2E2E', zorder=3)
    ax.coastlines(linewidth=0.3, zorder=4)
    ax.contourf(eofs.lon.values, eofs.lat.values, data,
                levels=11, vmin=-vm, vmax=vm,
                cmap=cmo.balance, transform=climpy.Map(), zorder=1)
    ax.set_title(title)
plt.tight_layout()
plt.show()
print('EOF1 mean pe bazin:', float(eofs.isel(mode=0).mean().round(4)))
print('EOF2 mean nord (lat>50):', float(eofs.isel(mode=1).sel(lat=slice(50,80)).mean().round(4)))
print('EOF2 mean sud (lat<30):', float(eofs.isel(mode=1).sel(lat=slice(0,30)).mean().round(4)))

### Identificare moduri

Uită-te la diagnostic de mai sus și ajustează coeficienții `a`, `b` de mai jos:
- **AMO** = modul cu pattern **uniform** pe tot bazinul
- **Tripole** = modul cu **3 centre** alternante (nord/centru/sud)

Dacă EOF1 = AMO: `a=1, b=1` pentru AMO și `a=1, b=-1` pentru Tripole  
Dacă EOF1 = Tripole: `a=1, b=-1` pentru AMO și `a=1, b=1` pentru Tripole  
Inversează semnele (`a=-1`) dacă pattern-ul e cu culorile invers față de așteptat.

In [ ]:
# ── Ajustează coeficienții după diagnosticul de mai sus ─────────────────────
eof_amo    = climpy.lincomb(eofs.isel(mode=0), eofs.isel(mode=1), a=1, b=-1)
eof_tripol = climpy.lincomb(eofs.isel(mode=0), eofs.isel(mode=1), a=1, b= 1)
pc_amo     = climpy.lincomb(pcs.isel(mode=0),  pcs.isel(mode=1),  a=1, b=-1)
pc_tripol  = climpy.lincomb(pcs.isel(mode=0),  pcs.isel(mode=1),  a=1, b= 1)

pc_amo_ma7    = climpy.moving_average(pc_amo,    n=7)
pc_tripol_ma7 = climpy.moving_average(pc_tripol, n=7)

print('PC-AMO dim:', pc_amo.dims)
print('PC-AMO first/last year:',
      pc_amo.time.dt.year.values[0], '–', pc_amo.time.dt.year.values[-1])

## 3 — Figura EOF

In [ ]:
fig = climpy.ClimPlot(
    nrows=2, ncols=2,
    w=climpy.NATURE_2COL, h=5.5,
    map_proj=(climpy.Map(), climpy.Map(), 'ts', 'ts'),
)
fig[0].map(eof_amo,
    title=f'EOF-AMO ({float(frac[0]):.1f} %)',
    map_extent=[-76, 20, 0, 80],
    vmin=-0.5, vmax=0.5, cbar_step=0.2, nlevels=11,
    cbar_label='SST anomaly (deg C)',
    cmap=CMAP_SST, show_land=True,
)
fig[1].map(eof_tripol,
    title=f'EOF-Tripole ({float(frac[1]):.1f} %)',
    map_extent=[-76, 20, 0, 80],
    vmin=-0.5, vmax=0.5, cbar_step=0.2, nlevels=11,
    cbar_label='SST anomaly (deg C)',
    cmap=CMAP_SST, show_land=True,
)
fig[2].ts(pc_amo, pc_amo_ma7,
    title='PC-AMO', ylabel='Amplitude',
    labels=['annual', '7-yr MA'],
    colors=['#0077BB', '#CC3311'],
)
fig[3].ts(pc_tripol, pc_tripol_ma7,
    title='PC-Tripole', ylabel='Amplitude',
    labels=['annual', '7-yr MA'],
    colors=['#0077BB', '#CC3311'],
)
fig.label_subplots()
fig.savefig('figures/fig_eof.pdf')
fig.show()

## 4 — Preprocessing SAT, SLP, Precipitații

In [ ]:
# SAT — GISTEMP
sat_r = climpy.preprocess(
    DATA / 'sat.nc', var='air',
    lon_convention='[-180,180]',
    time=('1880-01', '2015-12'),
    compute_annual=False, subtract_gm=False,
)
sat_djfm = climpy.seasonal_means(sat_r['anom'], months=[12,1,2,3]).dropna('year', how='all')
sat_jjas  = climpy.seasonal_means(sat_r['anom'], months=[6,7,8,9]).dropna('year', how='all')
print('SAT DJFM:', dict(sat_djfm.sizes))

# Precipitații — GPCC
pr_r = climpy.preprocess(
    DATA / 'precip.nc', var='precip',
    lon_convention='[-180,180]',
    time=('1891-01', '2015-12'),
    compute_annual=False, subtract_gm=False,
)
pr_djfm = climpy.seasonal_means(pr_r['anom'], months=[12,1,2,3]).dropna('year', how='all')
pr_jjas  = climpy.seasonal_means(pr_r['anom'], months=[6,7,8,9]).dropna('year', how='all')
print('Precip DJFM:', dict(pr_djfm.sizes))

# SLP — NCEP
slp_r = climpy.preprocess(
    DATA / 'slp.nc', var='slp',
    lon_convention='[-180,180]',
    time=('1948-01', '2024-12'),
    compute_annual=False, subtract_gm=False,
)
slp_djfm = climpy.seasonal_means(slp_r['anom'], months=[12,1,2,3]).dropna('year', how='all')
slp_djfm = slp_djfm / 100.0   # Pa -> hPa
print('SLP DJFM:', dict(slp_djfm.sizes))

## 5 — Corelații

In [ ]:
def align(pc, field):
    """Aliniere PC (dim=time datetime) cu câmp (dim=year int)."""
    if 'time' in pc.dims:
        pc_years = pc.time.dt.year.values
        pc = pc.assign_coords(year=('time', pc_years)).swap_dims({'time': 'year'})
    common = sorted(set(pc['year'].values) & set(field['year'].values))
    return pc.sel(year=common), field.sel(year=common)

# SAT DJFM
pc_a, sat_d  = align(pc_amo_ma7,    sat_djfm)
pc_t, sat_d2 = align(pc_tripol_ma7, sat_djfm)
r_amo_sat    = climpy.pearson_correlation(pc_a, sat_d,  dim='year')
r_tripol_sat = climpy.pearson_correlation(pc_t, sat_d2, dim='year')

# SLP DJFM
pc_a, slp_d  = align(pc_amo_ma7,    slp_djfm)
pc_t, slp_d2 = align(pc_tripol_ma7, slp_djfm)
r_amo_slp    = climpy.pearson_correlation(pc_a, slp_d,  dim='year')
r_tripol_slp = climpy.pearson_correlation(pc_t, slp_d2, dim='year')

# Precipitații DJFM
pc_a, pr_d   = align(pc_amo_ma7,    pr_djfm)
pc_t, pr_d2  = align(pc_tripol_ma7, pr_djfm)
r_amo_pr     = climpy.pearson_correlation(pc_a, pr_d,  dim='year')
r_tripol_pr  = climpy.pearson_correlation(pc_t, pr_d2, dim='year')

print('Corelații calculate.')

## 6 — Figura corelații (3×2: SAT / SLP / Precip × AMO / Tripole)

In [ ]:
fig = climpy.ClimPlot(
    nrows=3, ncols=2,
    w=climpy.NATURE_2COL, h=7.5,
    map_proj=(
        climpy.Map(-30), climpy.Map(-30),   # SAT row
        climpy.Map(-30), climpy.Map(-30),   # SLP row
        climpy.Map(-30), climpy.Map(-30),   # Precip row
    ),
)

BASE = dict(show_land=False, show_ocean=False,
            vmin=-0.6, vmax=0.6, cbar_step=0.2, nlevels=13, cbar_label='r')

# Rândul 1 — SAT (cmo.balance)
fig[0].map(r_amo_sat,    title='AMO x SAT (DJFM)',    cmap=CMAP_SAT,    **BASE)
fig[1].map(r_tripol_sat, title='Tripole x SAT (DJFM)', cmap=CMAP_SAT,    **BASE)

# Rândul 2 — SLP (cmo.curl)
fig[2].map(r_amo_slp,    title='AMO x SLP (DJFM)',    cmap=CMAP_SLP,    **BASE)
fig[3].map(r_tripol_slp, title='Tripole x SLP (DJFM)', cmap=CMAP_SLP,    **BASE)

# Rândul 3 — Precipitații (cmo.tarn)
fig[4].map(r_amo_pr,     title='AMO x Precip (DJFM)',    cmap=CMAP_PRECIP, **BASE)
fig[5].map(r_tripol_pr,  title='Tripole x Precip (DJFM)', cmap=CMAP_PRECIP, **BASE)

# Fond gri deschis pentru lizibilitate pe uscat
for ax in fig.axes:
    ax.set_facecolor('#F2F0EB')

fig.label_subplots()
fig.savefig('figures/fig_corr.pdf')
fig.show()

---
## Descărcare figuri

**Pe Binder:** click pe fișier în panoul Files → Download.

**Pe Colab:**
```python
from google.colab import files
files.download('figures/fig_eof.pdf')
files.download('figures/fig_corr.pdf')
```